In [93]:
# ====================================================
# Library
# ====================================================
import sys
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import timm

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop,
    HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
    Rotate, ShiftScaleRotate, CoarseDropout, Transpose 
)
from albumentations.pytorch import ToTensorV2

import warnings 
warnings.filterwarnings('ignore')

# Check available device
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")


device = get_device()
print(f"Using device: {device}")


NOTEBOOK_DIR = Path(os.getcwd())  
PROJECT_ROOT = NOTEBOOK_DIR.parent 


INPUT_DIR = PROJECT_ROOT / 'input' / 'ranzcr-clip-catheter-line-classification'
TRAIN_PATH = INPUT_DIR / 'train'
TEST_PATH = INPUT_DIR / 'test'
OUTPUT_DIR = NOTEBOOK_DIR / 'output-mask-batch-4-results' 
os.makedirs(OUTPUT_DIR, exist_ok=True)


def verify_paths():
    paths = {
        'Project Root': PROJECT_ROOT,
        'Input Directory': INPUT_DIR,
        'Train Path': TRAIN_PATH,
        'Test Path': TEST_PATH,
        'Output Directory': OUTPUT_DIR
    }
    
    print("Checking paths:")
    for name, path in paths.items():
        exists = path.exists()
        print(f"{name}: {path}")
        print(f"Exists: {exists}")
        if not exists:
            print(f"WARNING: {name} does not exist!")
    
  
    if TRAIN_PATH.exists():
        train_images = list(TRAIN_PATH.glob('*.jpg'))[:5] 
        print(f"\nFirst few training images:")
        for img_path in train_images:
            print(f"Image exists: {img_path.name} - {img_path.exists()}")

verify_paths()


# ====================================================
# Config
# ====================================================
class CFG:
    debug = False
    print_freq = 100
    num_workers = 0
    model_name = 'resnext50_32x4d'
    size = 512
    scheduler = 'CosineAnnealingLR'
    epochs = 30
    T_max = 6    
    lr = 1e-4
    min_lr = 1e-6
    batch_size = 4
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 11
    target_cols = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                   'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
                   'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                   'Swan Ganz Catheter Present']
    n_fold = 4
    trn_fold = [0]
    train = True
    train_path = str(TRAIN_PATH)
    test_path = str(TEST_PATH)
    output_dir = str(OUTPUT_DIR)
    
 
    use_masking = True
    mask_prob = 0.5
    
    intensity_threshold_levels = [30, 60, 90]

    anatomical_regions = {
        'central_width_ratio': 0.4,
        'upper_height_ratio': 0.4,
        'lateral_width_ratio': 0.2
    }

   
    patience = 3
    early_stopping = True


def clear_memory():
    import gc
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

seed_everything(CFG.seed)

Using device: mps
Checking paths:
Project Root: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT
Exists: True
Input Directory: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification
Exists: True
Train Path: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/train
Exists: True
Test Path: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/test
Exists: True
Output Directory: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/notebooks/output-mask-batch-4-results
Exists: True

First few training images:
Image exists: 1.2.826.0.1.3680043.8.498.16451034714945708059993280774682419855.jpg - True
Image exists: 1.2.826.0.1.3680043.8.498.20326719114358003969350032771972492089.jpg - True
Image exists: 1.2.826

In [94]:
class CombinedMask:
    @staticmethod
    def create_anatomical_mask(image_size):
        """Create anatomical mask with fixed size"""
        mask = np.zeros((image_size, image_size), dtype=np.uint8)
        
        central_width = int(image_size * CFG.anatomical_regions['central_width_ratio'])
        central_start = (image_size - central_width) // 2
        upper_height = int(image_size * CFG.anatomical_regions['upper_height_ratio'])
        lateral_width = int(image_size * CFG.anatomical_regions['lateral_width_ratio'])
        
        mask[:, central_start:central_start + central_width] = 1
        mask[:upper_height, :] = 1
        mask[:, :lateral_width] = 1
        mask[:, -lateral_width:] = 1
        
        return mask

    @staticmethod
    def create_intensity_mask(image):
        if image is None:
            return None
            
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        masks = []
        
        for level in CFG.intensity_threshold_levels:
            _, thresh = cv2.threshold(gray, level, 255, cv2.THRESH_BINARY)
            masks.append(thresh)
       
        final_mask = np.zeros_like(masks[0])
        for mask in masks:
            final_mask = cv2.bitwise_or(final_mask, mask)
               
        kernel = np.ones((5,5), np.uint8)
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, kernel)
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_OPEN, kernel)
        
        return final_mask

    @staticmethod
    def apply_mask(image):
        if random.random() < CFG.mask_prob:
            image = cv2.resize(image, (CFG.size, CFG.size))
            
            # Create anatomical mask with fixed size
            anatomical_mask = CombinedMask.create_anatomical_mask(CFG.size)
            # Create intensity mask on resized image
            intensity_mask = CombinedMask.create_intensity_mask(image)
            
            if intensity_mask is not None:
                combined_mask = (anatomical_mask * 0.7 + (intensity_mask/255.0) * 0.3)
                combined_mask = (combined_mask > 0.5).astype(np.uint8)
                combined_mask = np.stack([combined_mask] * 3, axis=-1)
                
                masked_image = np.where(combined_mask == 1, image, image * 0.3)
                return masked_image
        return image

In [95]:
# ====================================================
# Dataset 
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.labels = df[CFG.target_cols].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            file_name = self.file_names[idx]
            file_path = Path(CFG.train_path) / f'{file_name}.jpg' 

            if not file_path.exists():
                print(f"File not found: {file_path}")
                image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            else:
                image = cv2.imread(str(file_path))
                if image is None:
                    print(f"Failed to read image: {file_path}")
                    image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
                else:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                   
                if CFG.use_masking:
                    image = CombinedMask.apply_mask(image)
               
                elif image.shape[:2] != (CFG.size, CFG.size):
                    image = cv2.resize(image, (CFG.size, CFG.size))

            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']

            label = torch.tensor(self.labels[idx]).float()
            return image, label

        except Exception as e:
            print(f"Error loading image {file_name} at index {idx}: {str(e)}")
            image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            label = torch.tensor(self.labels[idx]).float()
            return image, label

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = Path(CFG.test_path) / f'{file_name}.jpg'
        
        if not file_path.exists():
            print(f"Test file not found: {file_path}")
            image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
        else:
            image = cv2.imread(str(file_path))
            if image is None:
                print(f"Failed to read test image: {file_path}")
                image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            if CFG.use_masking:
                image = CombinedMask.apply_mask(image)
            # Ensure image is resized if masking wasn't applied
            elif image.shape[:2] != (CFG.size, CFG.size):
                image = cv2.resize(image, (CFG.size, CFG.size))
                
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image

In [96]:
# ====================================================
# Model Architecture
# ====================================================

class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features  

    
        self.model.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(n_features, CFG.target_size)
        )

    def forward(self, x):
        x = self.model(x)
        return x
    
# ====================================================
# Data Transforms
# ====================================================
def get_transforms(*, data):
    if data == 'train':
        return Compose([
            RandomResizedCrop(CFG.size, CFG.size, scale=(0.85, 1.0)),
            HorizontalFlip(p=0.5),
            RandomBrightnessContrast(p=0.5, brightness_limit=0.2, contrast_limit=0.2),
            ShiftScaleRotate(p=0.5, shift_limit=0.2, scale_limit=0.2, rotate_limit=20),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# ====================================================
# Metrics
# ====================================================
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = roc_auc_score(y_true[:, i], y_pred[:, i])
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(outputs, targets):
    with torch.no_grad():
        predictions = (outputs.sigmoid() > 0.5).float()
        correct = (predictions == targets).float().sum()
        total = targets.numel()
        accuracy = correct / total
    return accuracy.item()

# ====================================================
# Training Functions
# ====================================================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    train_loss = AverageMeter()
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Train')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        optimizer.zero_grad()
        
        y_preds = model(images)
        loss = criterion(y_preds, labels)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), 
            CFG.max_grad_norm
        )
        
        optimizer.step()
        
        train_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix(**{
                'train_loss': train_loss.avg,
                'grad_norm': grad_norm.item(),
                'lr': optimizer.param_groups[0]['lr']
            })
        
        if step % 10 == 0:
            clear_memory()
            
    return train_loss.avg


def valid_one_epoch(model, loader, criterion, device):
    model.eval()
    valid_loss = AverageMeter()
    valid_acc = AverageMeter()  
    predictions = []
    targets = []
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Valid')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            
            acc = calculate_accuracy(y_preds, labels)
            valid_acc.update(acc, batch_size)
        
        predictions.append(y_preds.sigmoid().cpu().numpy())
        targets.append(labels.cpu().numpy())
        
        valid_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix({
                'valid_loss': valid_loss.avg,
                'valid_acc': valid_acc.avg
            })
    
    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    
    return valid_loss.avg, valid_acc.avg, predictions, targets


In [97]:

# ====================================================
# Training Functions
# ====================================================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    train_loss = AverageMeter()
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Train')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        optimizer.zero_grad()
        
        y_preds = model(images)
        loss = criterion(y_preds, labels)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), 
            CFG.max_grad_norm
        )
        
        optimizer.step()
        
        train_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix(**{
                'train_loss': train_loss.avg,
                'grad_norm': grad_norm.item(),
                'lr': optimizer.param_groups[0]['lr']
            })
        
        if step % 10 == 0:
            clear_memory()
            
    return train_loss.avg

def valid_one_epoch(model, loader, criterion, device):
    model.eval()
    valid_loss = AverageMeter()
    valid_acc = AverageMeter() 
    predictions = []
    targets = []
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Valid')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            
            acc = calculate_accuracy(y_preds, labels)
            valid_acc.update(acc, batch_size)
        
        predictions.append(y_preds.sigmoid().cpu().numpy())
        targets.append(labels.cpu().numpy())
        
        valid_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix({
                'valid_loss': valid_loss.avg,
                'valid_acc': valid_acc.avg
            })
    
    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    
    return valid_loss.avg, valid_acc.avg, predictions, targets

In [98]:
class Logger:
    def __init__(self, output_dir, fold):
        """
        Initialize Logger with output directory and fold number
        
        Args:
            output_dir (str): Directory to save logs
            fold (int): Fold number for cross validation
        """
        self.output_dir = output_dir
        self.fold = fold
        
        self.log_file = f'{output_dir}/training_log_fold{fold}.csv'
        self.predictions_file = f'{output_dir}/predictions_fold{fold}.csv'
        self.best_model_log = f'{output_dir}/best_model_fold{fold}.txt'
        
        self._initialize_log_file()
        
        self.best_score = 0.0
        self.best_epoch = 0
        
    def _initialize_log_file(self):
        """Initialize the CSV log file with headers"""
        with open(self.log_file, 'w') as f:
            headers = [
                'epoch',
                'learning_rate',
                'train_loss',
                'train_accuracy',
                'valid_loss',
                'valid_accuracy',
                'auc_score'
            ]
            headers.extend([f'auc_{col}' for col in CFG.target_cols])
            f.write(','.join(headers) + '\n')

    def log_epoch(self, epoch, lr, train_metrics, valid_metrics, predictions, targets):
        """
        Log metrics for current epoch
        
        Args:
            epoch (int): Current epoch number
            lr (float): Current learning rate
            train_metrics (dict): Training metrics
            valid_metrics (dict): Validation metrics
            predictions (np.array): Model predictions
            targets (np.array): True labels
        """
        # Calculate per-class AUC scores
        avg_auc_score, class_auc_scores = get_score(targets, predictions)
        
        metrics = [
            str(epoch),
            f'{lr:.8f}',
            f'{train_metrics["loss"]:.6f}',
            f'{train_metrics["accuracy"]:.6f}',
            f'{valid_metrics["loss"]:.6f}',
            f'{valid_metrics["accuracy"]:.6f}',
            f'{avg_auc_score:.6f}'
        ]
        
        metrics.extend([f'{score:.6f}' for score in class_auc_scores])
        
        with open(self.log_file, 'a') as f:
            f.write(','.join(metrics) + '\n')
        
        if epoch % 5 == 0:
            self.save_predictions(epoch, predictions, targets)
            
        return avg_auc_score, class_auc_scores

    def save_predictions(self, epoch, predictions, targets):
        """
        Save model predictions and true labels
        
        Args:
            epoch (int): Current epoch
            predictions (np.array): Model predictions
            targets (np.array): True labels
        """
        pred_df = pd.DataFrame(predictions, columns=[f'pred_{col}' for col in CFG.target_cols])
        true_df = pd.DataFrame(targets, columns=[f'true_{col}' for col in CFG.target_cols])
        
        results_df = pd.concat([pred_df, true_df], axis=1)
        
        save_path = f'{self.output_dir}/predictions_fold{self.fold}_epoch{epoch}.csv'
        results_df.to_csv(save_path, index=False)
        print(f"Saved predictions to {save_path}")

    def log_best_model(self, epoch, score, model_path, class_scores, model_state=None):
        """
        Log best model details and save model state
        
        Args:
            epoch (int): Epoch number
            score (float): Model score
            model_path (str): Path where model is saved
            class_scores (list): Individual class scores
            model_state (dict, optional): Model state dictionary
        """
        if score > self.best_score:
            self.best_score = score
            self.best_epoch = epoch
            
            with open(self.best_model_log, 'w') as f:
                f.write(f"Best Model Details:\n")
                f.write(f"Epoch: {epoch}\n")
                f.write(f"Overall Score: {score:.6f}\n")
                f.write("\nClass-wise scores:\n")
                for class_name, class_score in zip(CFG.target_cols, class_scores):
                    f.write(f"{class_name}: {class_score:.6f}\n")
                f.write(f"\nModel saved at: {model_path}\n")
            
            if model_state is not None:
                torch.save(model_state, model_path)
                
            print(f"\nNew best model saved!")
            print(f"Overall Score: {score:.4f}")
            print("\nClass-wise scores:")
            for class_name, class_score in zip(CFG.target_cols, class_scores):
                print(f"{class_name}: {class_score:.4f}")

    def get_training_summary(self):
        """
        Generate training summary from logs
        
        Returns:
            dict: Summary of training metrics
        """
        try:
            df = pd.read_csv(self.log_file)
            summary = {
                'best_epoch': self.best_epoch,
                'best_score': self.best_score,
                'final_train_loss': df['train_loss'].iloc[-1],
                'final_valid_loss': df['valid_loss'].iloc[-1],
                'final_train_acc': df['train_accuracy'].iloc[-1],
                'final_valid_acc': df['valid_accuracy'].iloc[-1],
                'best_auc_score': df['auc_score'].max(),
                'class_scores': {
                    col: df[f'auc_{col}'].max() 
                    for col in CFG.target_cols
                }
            }
            return summary
        except Exception as e:
            print(f"Error generating summary: {str(e)}")
            return None

def plot_training_curves(self, fold):
    """
    Plot and save training curves for loss, accuracy, and AUC
    
    Args:
        fold (int): Fold number for saving plots
    """
    try:
        df = pd.read_csv(self.log_file)
        
        plots_dir = Path(self.output_dir) / 'plots'
        plots_dir.mkdir(exist_ok=True)
        
        plt.figure(figsize=(10, 6))
        plt.plot(df['epoch'], df['train_loss'], label='Train Loss', marker='o')
        plt.plot(df['epoch'], df['valid_loss'], label='Valid Loss', marker='o')
        plt.title(f'Fold {fold} - Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.savefig(plots_dir / f'fold{fold}_loss.png')
        plt.close()
        
        plt.figure(figsize=(10, 6))
        plt.plot(df['epoch'], df['train_accuracy'], label='Train Accuracy', marker='o')
        plt.plot(df['epoch'], df['valid_accuracy'], label='Valid Accuracy', marker='o')
        plt.title(f'Fold {fold} - Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        plt.savefig(plots_dir / f'fold{fold}_accuracy.png')
        plt.close()
        
        plt.figure(figsize=(10, 6))
        plt.plot(df['epoch'], df['auc_score'], label='AUC Score', marker='o')
        plt.title(f'Fold {fold} - AUC Score')
        plt.xlabel('Epoch')
        plt.ylabel('AUC')
        plt.legend()
        plt.grid(True)
        plt.savefig(plots_dir / f'fold{fold}_auc.png')
        plt.close()
        
        plt.figure(figsize=(15, 8))
        for col in CFG.target_cols:
            plt.plot(df['epoch'], df[f'auc_{col}'], label=col, marker='o')
        plt.title(f'Fold {fold} - Class-wise AUC Scores')
        plt.xlabel('Epoch')
        plt.ylabel('AUC')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.grid(True)
        plt.savefig(plots_dir / f'fold{fold}_class_auc.png')
        plt.close()
        
        print(f"Training curves saved in {plots_dir}")
        
    except Exception as e:
        print(f"Error plotting curves: {str(e)}")

In [99]:
def train_loop(train_loader, valid_loader, model, criterion, optimizer, scheduler, fold):
    logger = Logger(CFG.output_dir, fold)
    best_score = 0.
    best_loss = np.inf
    patience_counter = 0
    best_scores_array = None  # To store class-wise scores for best model
    
    for epoch in range(CFG.epochs):
        print(f'Epoch {epoch+1}/{CFG.epochs}')
        
        # Training phase
        model.train()
        train_loss = AverageMeter()
        train_acc = AverageMeter()
        
        for images, labels in tqdm(train_loader, desc='Train'):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            acc = calculate_accuracy(outputs, labels)
            train_loss.update(loss.item(), labels.size(0))
            train_acc.update(acc, labels.size(0))
            
        # Validation phase
        valid_loss, valid_acc, predictions, targets = valid_one_epoch(
            model, valid_loader, criterion, device
        )
        
        # Calculate overall and per-class scores
        score, class_scores = get_score(targets, predictions)
        
        # Update scheduler
        current_lr = optimizer.param_groups[0]['lr']
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(valid_loss)
        else:
            scheduler.step()
        
        # Log metrics
        train_metrics = {
            'loss': train_loss.avg,
            'accuracy': train_acc.avg
        }
        valid_metrics = {
            'loss': valid_loss,
            'accuracy': valid_acc
        }
        
        logger.log_epoch(
            epoch + 1,
            current_lr,
            train_metrics,
            valid_metrics,
            predictions,
            targets
        )
        
        # Print current class-wise scores
        print("\nClass-wise AUC scores:")
        for class_name, class_score in zip(CFG.target_cols, class_scores):
            print(f"{class_name}: {class_score:.4f}")
        
        # Save best model and scores
        if score > best_score:
            best_score = score
            best_loss = valid_loss
            best_scores_array = class_scores  # Store class-wise scores
            patience_counter = 0
            
            model_path = f'{CFG.output_dir}/best_fold{fold}.pth'
            torch.save({
                'model': model.state_dict(),
                'predictions': predictions,
                'targets': targets,
                'epoch': epoch,
                'score': best_score,
                'class_scores': class_scores, 
                'target_cols': CFG.target_cols  
            }, model_path)
            
            logger.log_best_model(epoch + 1, best_score, model_path, class_scores)
            print(f'\nNew best model saved!')
            print(f'Overall Score: {best_score:.4f}')
            print("\nBest model class-wise scores:")
            for class_name, class_score in zip(CFG.target_cols, class_scores):
                print(f"{class_name}: {class_score:.4f}")
        else:
            patience_counter += 1
        
        # Early stopping check
        if CFG.early_stopping and patience_counter >= CFG.patience:
            print(f'Early stopping triggered after {epoch + 1} epochs')
            break
        
        clear_memory()
        
    
    return best_score, best_loss, best_scores_array

In [100]:

# ====================================================
# Main Training Function
# ====================================================
def train_model():
    train_csv_path = INPUT_DIR / 'train.csv'
    test_csv_path = INPUT_DIR / 'sample_submission.csv'
    
    print(f"Loading data from: {train_csv_path}")
    train = pd.read_csv(train_csv_path)
    test = pd.read_csv(test_csv_path)
    
    if CFG.debug:
        train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)
    
    
    Fold = GroupKFold(n_splits=4)
    groups = train['PatientID'].values
    for n, (train_index, valid_index) in enumerate(Fold.split(train, train[CFG.target_cols], groups)):
        train.loc[valid_index, 'fold'] = int(n)
    train['fold'] = train['fold'].astype(int)
    
    for fold in CFG.trn_fold: 
        print(f'Training fold {fold}')
        
        train_df = train[train.fold != fold].reset_index(drop=True)
        valid_df = train[train.fold == fold].reset_index(drop=True)
        
        train_dataset = TrainDataset(train_df, transform=get_transforms(data='train'))
        valid_dataset = TrainDataset(valid_df, transform=get_transforms(data='valid'))
        
        train_loader = DataLoader(
            train_dataset,
            batch_size=CFG.batch_size,
            shuffle=True,
            num_workers=CFG.num_workers,
            pin_memory=True,
            drop_last=True
        )
        
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=CFG.batch_size,
            shuffle=False,
            num_workers=CFG.num_workers,
            pin_memory=True,
            drop_last=False
        )
        
        model = CustomResNext(CFG.model_name, pretrained=True)
        model.to(device)
        
        optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr)
        criterion = nn.BCEWithLogitsLoss()
        
        best_score, best_loss, class_scores = train_loop(
            train_loader,
            valid_loader,
            model,
            criterion,
            optimizer,
            scheduler,
            fold
        )
        
        print(f'Fold {fold} Results:')
        print(f'Best Score: {best_score:.4f}')
        print(f'Best Loss: {best_loss:.4f}')
        print('\nClass-wise scores:')

        for class_name, score in zip(CFG.target_cols, class_scores):
            print(f'{class_name}: {score:.4f}')

        clear_memory()

In [101]:
def predict_test():
    print('Loading test data...')
    test_csv_path = INPUT_DIR / 'sample_submission.csv'
    test_df = pd.read_csv(test_csv_path)
    
    # Create test dataset
    test_dataset = TestDataset(
        test_df, 
        transform=get_transforms(data='valid')
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True
    )
    
    models = []
    for fold in CFG.trn_fold:
        model = CustomResNext(CFG.model_name, pretrained=False)
        model_path = f'{OUTPUT_DIR}/best_fold{fold}.pth'
        model.load_state_dict(torch.load(model_path)['model'])
        model.to(device)
        model.eval()
        models.append(model)
    
    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader, desc='Predict'):
            images = images.to(device)
            outputs = None
            
            for model in models:
                if outputs is None:
                    outputs = model(images).sigmoid()
                else:
                    outputs += model(images).sigmoid()
            outputs /= len(models)
            
            predictions.append(outputs.cpu().numpy())
    
    predictions = np.concatenate(predictions)
    
    submission = pd.DataFrame(data=predictions, columns=CFG.target_cols)
    submission['StudyInstanceUID'] = test_df['StudyInstanceUID']
    submission.to_csv(INPUT_DIR / 'sample_submission.csv', index=False)
    print('Predictions saved to sample_submission.csv')

In [102]:
if __name__ == '__main__':
    print('Training started...')
    seed_everything(CFG.seed)
    train_model()
    print('Training completed!')
    
    print('\nGenerating test predictions...')
    predict_test()
    print('All done!')

Training started...
Loading data from: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/train.csv
Training fold 0
Epoch 1/30


Train:   0%|          | 0/5640 [00:00<?, ?it/s]

Valid:   0%|          | 0/1881 [00:00<?, ?it/s]


Class-wise AUC scores:
ETT - Abnormal: 0.8746
ETT - Borderline: 0.8684
ETT - Normal: 0.9733
NGT - Abnormal: 0.7478
NGT - Borderline: 0.7972
NGT - Incompletely Imaged: 0.8895
NGT - Normal: 0.9009
CVC - Abnormal: 0.5649
CVC - Borderline: 0.5796
CVC - Normal: 0.5493
Swan Ganz Catheter Present: 0.8308

New best model saved!
Overall Score: 0.7797

Class-wise scores:
ETT - Abnormal: 0.8746
ETT - Borderline: 0.8684
ETT - Normal: 0.9733
NGT - Abnormal: 0.7478
NGT - Borderline: 0.7972
NGT - Incompletely Imaged: 0.8895
NGT - Normal: 0.9009
CVC - Abnormal: 0.5649
CVC - Borderline: 0.5796
CVC - Normal: 0.5493
Swan Ganz Catheter Present: 0.8308

New best model saved!
Overall Score: 0.7797

Best model class-wise scores:
ETT - Abnormal: 0.8746
ETT - Borderline: 0.8684
ETT - Normal: 0.9733
NGT - Abnormal: 0.7478
NGT - Borderline: 0.7972
NGT - Incompletely Imaged: 0.8895
NGT - Normal: 0.9009
CVC - Abnormal: 0.5649
CVC - Borderline: 0.5796
CVC - Normal: 0.5493
Swan Ganz Catheter Present: 0.8308
Epoch 2

Train:   0%|          | 0/5640 [00:00<?, ?it/s]

...

Train:   0%|          | 0/5640 [00:00<?, ?it/s]

Valid:   0%|          | 0/1881 [00:00<?, ?it/s]


Class-wise AUC scores:
ETT - Abnormal: 0.9455
ETT - Borderline: 0.9458
ETT - Normal: 0.9888
NGT - Abnormal: 0.8589
NGT - Borderline: 0.9074
NGT - Incompletely Imaged: 0.9614
NGT - Normal: 0.9663
CVC - Abnormal: 0.8719
CVC - Borderline: 0.7807
CVC - Normal: 0.8632
Swan Ganz Catheter Present: 0.9868
Early stopping triggered after 22 epochs
Fold 0 Results:
Best Score: 0.9279
Best Loss: 0.1592

Class-wise scores:
ETT - Abnormal: 0.9644
ETT - Borderline: 0.9526
ETT - Normal: 0.9907
NGT - Abnormal: 0.8808
NGT - Borderline: 0.9156
NGT - Incompletely Imaged: 0.9656
NGT - Normal: 0.9718
CVC - Abnormal: 0.8845
CVC - Borderline: 0.8059
CVC - Normal: 0.8811
Swan Ganz Catheter Present: 0.9937
Training completed!

Generating test predictions...
Loading test data...


Predict:   0%|          | 0/896 [00:00<?, ?it/s]

Predictions saved to sample_submission.csv
All done!
